Load Packages

In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, confusion_matrix, mean_squared_error

In [21]:
# Load in csv
df = pd.read_csv('C:/Users/tmanett/Desktop/Manifest Optimization/TML Manifest Data.csv')
df.index = pd.to_datetime(df['event_date'])
df.head()

C:\Users\tmanett\AppData\Local\Temp\ipykernel_27200\3307145020.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/tmanett/Desktop/Manifest Optimization/TML Manifest Data.csv')


,Unnamed: 0,event_date,event_name,section_name,row_name,seat_num,activity_name,fiscal,tier,def_price_code,...,section_index,row_index,team,price_code_desc,price_diff_perc,price_diff_doll,resale_price,face_value,resale_price_outlier,price_diff_perc_outlier
event_date,,,,,,,,,,,,,,,,,,,,,
2016-02-18,1,2/18/2016,L160218,101,15,17,TE Resale,F16,5,F,...,0.517241,0.500000,New York Rangers,Light Gold,-0.059406,-12.0,190.0,202,no,no
2016-02-18,2,2/18/2016,L160218,101,15,18,TE Resale,F16,5,F,...,0.517241,0.529412,New York Rangers,Light Gold,-0.059406,-12.0,190.0,202,no,no
2016-02-18,3,2/18/2016,L160218,101,9,1,TE Resale,F16,5,C,...,0.310345,0.032258,New York Rangers,Plat Ends,-0.087678,-18.5,192.5,211,no,no
2016-02-18,4,2/18/2016,L160218,101,9,2,TE Resale,F16,5,C,...,0.310345,0.064516,New York Rangers,Plat Ends,-0.087678,-18.5,192.5,211,no,no
2016-02-25,5,2/25/2016,L160225,101,27,13,TE Resale,F16,5,I,...,0.931034,0.481481,Carolina Hurricanes,Club Ends,0.235669,37.0,194.0,157,no,no


Remove columns that will not be used in model

In [22]:
df.drop(['Unnamed: 0', 'event_date', 'event_name', 'activity_name', 'fiscal', 'def_price_code', 'quality', 'direction', 'player_tunnel', 'section_index', 'row_index'], axis=1, inplace=True)

In [23]:
df.head()

,section_name,row_name,seat_num,tier,bowl_level,view_level,side_level,row_number,section_number,penalty_box,...,seat_class,aisle,team,price_code_desc,price_diff_perc,price_diff_doll,resale_price,face_value,resale_price_outlier,price_diff_perc_outlier
event_date,,,,,,,,,,,,,,,,,,,,,
2016-02-18,101,15,17,5,Lower Bowl,Corners,Defend Side,15,101,False,...,STANDARD,True,New York Rangers,Light Gold,-0.059406,-12.0,190.0,202,no,no
2016-02-18,101,15,18,5,Lower Bowl,Corners,Defend Side,15,101,False,...,STANDARD,True,New York Rangers,Light Gold,-0.059406,-12.0,190.0,202,no,no
2016-02-18,101,9,1,5,Lower Bowl,Corners,Defend Side,9,101,False,...,OBSTRUCTED,True,New York Rangers,Plat Ends,-0.087678,-18.5,192.5,211,no,no
2016-02-18,101,9,2,5,Lower Bowl,Corners,Defend Side,9,101,False,...,OBSTRUCTED,True,New York Rangers,Plat Ends,-0.087678,-18.5,192.5,211,no,no
2016-02-25,101,27,13,5,Lower Bowl,Corners,Defend Side,27,101,False,...,STANDARD,False,Carolina Hurricanes,Club Ends,0.235669,37.0,194.0,157,no,no


Remove whitespace 

In [25]:
# Assuming df is your DataFrame
columns_to_replace = ['bowl_level', 'side_level', 'team', 'price_code_desc']
df[columns_to_replace] = df[columns_to_replace].replace(' ', '_', regex=True)

# Print the first 5 rows of the DataFrame to check the modifications
print(df.head(5))

           section_name row_name  seat_num  tier  bowl_level view_level   
event_date                                                                
2016-02-18          101       15        17     5  Lower_Bowl    Corners  \
2016-02-18          101       15        18     5  Lower_Bowl    Corners   
2016-02-18          101        9         1     5  Lower_Bowl    Corners   
2016-02-18          101        9         2     5  Lower_Bowl    Corners   
2016-02-25          101       27        13     5  Lower_Bowl    Corners   

             side_level  row_number  section_number  penalty_box  ...   
event_date                                                        ...   
2016-02-18  Defend_Side          15             101        False  ...  \
2016-02-18  Defend_Side          15             101        False  ...   
2016-02-18  Defend_Side           9             101        False  ...   
2016-02-18  Defend_Side           9             101        False  ...   
2016-02-25  Defend_Side          27 

# Missing Data

In [26]:
df.dtypes

section_name                object
row_name                    object
seat_num                     int64
tier                         int64
bowl_level                  object
view_level                  object
side_level                  object
row_number                   int64
section_number               int64
penalty_box                   bool
bench                         bool
seat_class                  object
aisle                         bool
team                        object
price_code_desc             object
price_diff_perc            float64
price_diff_doll            float64
resale_price               float64
face_value                   int64
resale_price_outlier        object
price_diff_perc_outlier     object
dtype: object

In [27]:
# Assuming df is your DataFrame
missing_data = df.isnull().sum()

print(missing_data)

section_name               0
row_name                   0
seat_num                   0
tier                       0
bowl_level                 0
view_level                 0
side_level                 0
row_number                 0
section_number             0
penalty_box                0
bench                      0
seat_class                 0
aisle                      0
team                       0
price_code_desc            0
price_diff_perc            0
price_diff_doll            0
resale_price               0
face_value                 0
resale_price_outlier       0
price_diff_perc_outlier    0
dtype: int64


Split Data into Dependent and Independent

In [29]:
X = df.drop('price_diff_perc', axis=1).copy()
X.head()

,section_name,row_name,seat_num,tier,bowl_level,view_level,side_level,row_number,section_number,penalty_box,bench,seat_class,aisle,team,price_code_desc,price_diff_doll,resale_price,face_value,resale_price_outlier,price_diff_perc_outlier
event_date,,,,,,,,,,,,,,,,,,,,
2016-02-18,101,15,17,5,Lower_Bowl,Corners,Defend_Side,15,101,False,False,STANDARD,True,New_York_Rangers,Light_Gold,-12.0,190.0,202,no,no
2016-02-18,101,15,18,5,Lower_Bowl,Corners,Defend_Side,15,101,False,False,STANDARD,True,New_York_Rangers,Light_Gold,-12.0,190.0,202,no,no
2016-02-18,101,9,1,5,Lower_Bowl,Corners,Defend_Side,9,101,False,False,OBSTRUCTED,True,New_York_Rangers,Plat_Ends,-18.5,192.5,211,no,no
2016-02-18,101,9,2,5,Lower_Bowl,Corners,Defend_Side,9,101,False,False,OBSTRUCTED,True,New_York_Rangers,Plat_Ends,-18.5,192.5,211,no,no
2016-02-25,101,27,13,5,Lower_Bowl,Corners,Defend_Side,27,101,False,False,STANDARD,False,Carolina_Hurricanes,Club_Ends,37.0,194.0,157,no,no


In [30]:
y = df['price_diff_perc'].copy()
y.head()

event_date
2016-02-18   -0.059406
2016-02-18   -0.059406
2016-02-18   -0.087678
2016-02-18   -0.087678
2016-02-25    0.235669
Name: price_diff_perc, dtype: float64

### One-Hot Encoding

- section_name: Categorical
- row_name: Categorical
- seat_num: Categorical
- tier: Categorical
- bowl_level: Categorical
- view_level: Categorical
- side_level: Categorical
- row_number: Categorical
- section_number: Categorical
- penalty_box: Boolean
- bench: Boolean
- seat_class: Categorical
- aisle: Boolean
- team: Categorical
- price_code_desc: Categorical
- price_diff_doll: Float (continuous)
- resale_price: Float (continuous)
- face_value: Float (continuous)
- resale_price_outlier: Boolean
- price_diff_perc_outlier: Boolean

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error

cat_cols = ['section_name', 'row_name', 'bowl_level', 'view_level', 'side_level', 'seat_class', 'team', 'price_code_desc', 'resale_price_outlier', 'price_diff_perc_outlier']


# Convert columns to string data type
X[cat_cols] = X[cat_cols].astype(str)

X_encoded = X.copy()
encoder = LabelEncoder()
for col in cat_cols:
    X_encoded[col] = encoder.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create an XGBoost regressor object
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)

# Fit the XGBoost model to the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.036911210117712576
